In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [58]:
#import the necessary libraries
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

#define the batch size for training
batch_size =  64 
#define the number of epoch for training
epochs = 100 
#define the dimensionality of the encoding process
latent_dim = 256 
#define the number of samples to train on
num_samples = 10000

In [3]:
#define the file location
data_path = "/content/drive/MyDrive/Colab Notebooks/vie.txt"
#define an empty list to store the words/sentences in English
input_texts = []
#define an empty list to store the characters in Spanish
target_texts = []
#define a set to store the words/sentences in English
#a set data type and not a list is used to avoid repetition of characters
input_characters = set()
#define a set to store the characters in Spanish
target_characters = set()
#read the data file and parse by each line
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    #split each line into the input text, target text and the other unnecessary text
    input_text, target_text, _ = line.split('\t')
    #we use tab as the start sequence character for the target, and \n as end sequence character
    target_text = '\t' + target_text + '\n'
    #append the English words for each text to the empty list
    input_texts.append(input_text)
    #append the Spanish words for each text to the empty list
    target_texts.append(target_text)
    #select the letters or words in English
    for char in input_text:
        #check if the letter is not in the list
        if char not in input_characters:
            #append the letter that is not on the list
            input_characters.add(char)
    #select the letters or words in Spanish
    for char in target_text:
        #check if the letter or word is not in the list
        if char not in target_characters:
            #append the letter or word that is not on the list
            target_characters.add(char)

In [4]:
print(input_characters)

{' ', 'l', '.', 'C', 'T', '9', 'd', 'H', 'S', ',', 'x', 'b', '8', '$', 'U', 'P', 'u', 'B', 'o', 'R', 'v', '!', 'n', 'I', 'Y', 'V', 'M', 'i', '1', '6', '%', '-', '?', 'W', '7', 't', '0', 'j', 'e', 'a', 'F', 'q', "'", 's', '"', '5', 'A', 'K', 'f', 'E', 'p', 'y', 'O', 'z', 'r', 'N', '2', '3', '’', 'k', 'Q', ':', 'g', 'w', '4', 'h', 'c', 'J', '+', 'L', 'm', '€', 'G', 'D'}


In [5]:
print(target_characters)

{'ợ', '.', 'ặ', 'ỳ', 'ộ', 'Ư', 'É', 'u', 'U', 'ủ', 'ó', 'ò', 'ố', '6', 'ẳ', 'ầ', 'ẻ', 'Ồ', 'ậ', 'Ý', 'ẵ', 'q', 'Ấ', 'ớ', '"', 'à', '5', 'ú', 'K', 'ể', '2', 'Ổ', 'ổ', 'ỗ', 'ẩ', 'đ', 'w', 'h', 'c', 'ơ', 'm', 'ễ', 'ề', 'D', 'Đ', ' ', 'Ả', 'Ờ', 'Ă', 'ư', 'ệ', 'H', 'á', 'b', 'P', '>', 'B', 'o', 'ị', 'M', 'Ứ', 'ở', 'ẫ', 'Â', 'ỷ', 'â', 'ử', 'j', 'ứ', 'õ', 'a', 'ì', 'ù', 'Ở', 'ỵ', 'ã', 'f', 'y', 'X', 'ằ', 'ý', '3', 'ẽ', 'Ú', '4', 'ự', 'G', '(', 'ữ', 'l', 'C', 'T', 'd', 'x', '8', '$', '-', 'í', '\n', 'n', 'Y', 'I', 'i', '1', '%', 'ũ', 'ỹ', 'ê', 'ắ', 'e', 'ấ', 's', 'è', 'ừ', 'A', 'ỉ', 'ạ', 'ồ', 'O', 'z', 'N', 'r', 'ẹ', ')', 'J', 'Á', 'L', 'ỏ', 'ô', '/', '9', 'ụ', 'S', ',', 'R', 'v', '!', 'V', 'é', 'Ô', '?', 'Í', 'W', '7', 't', '0', 'F', 'ờ', 'ĩ', 'ỡ', 'E', 'p', 'ă', 'k', 'Q', 'ọ', ':', 'g', '+', '€', 'ế', '\t', 'ả'}


In [6]:
print(input_texts[: 50])

['Run!', 'Help!', 'Go on.', 'Hello!', 'Hurry!', 'I try.', 'I try.', 'Eat it.', 'Eat it.', 'Got it!', 'Help me!', 'I agree.', 'Perfect!', 'We know.', 'We know.', 'You run.', 'Cheer up.', 'He tries.', 'He tries.', 'Hurry up.', 'I forgot.', "I'll try.", "I'll try.", "I'm bald.", "I'm busy.", 'Too late.', 'Come here.', 'Come here.', 'Come here.', 'Go to bed.', 'I hate TV.', 'I laughed.', 'I laughed.', 'I will go.', 'Is it bad?', "It's cold.", "It's ours.", "It's ours.", 'Keep back.', 'Keep back.', 'Look here.', 'She cried.', 'She cried.', 'Sit there.', 'Stay home.', "What's up?", "What's up?", 'Are you OK?', 'Find a job.', 'Hurry home.']


In [7]:
print(target_texts[:50])

['\tChạy!\n', '\tGiúp tôi với!\n', '\tTiếp tục đi.\n', '\tChào bạn.\n', '\tNhanh lên nào!\n', '\tTôi sẽ thử.\n', '\tTôi sẽ thử xem sao.\n', '\tĂn đi.\n', '\tĂn nó đi.\n', '\tHiểu rồi!\n', '\tCứu tôi với!\n', '\tTôi cũng nghĩ như vậy.\n', '\tHoàn hảo!\n', '\tChúng tôi biết.\n', '\tChúng ta biết.\n', '\tBạn chạy.\n', '\tĐừng có rầu rĩ quá như thế.\n', '\tHắn thử.\n', '\tAnh thử.\n', '\tThoáng cái chân lên!\n', '\tTôi quên mất rồi.\n', '\tTôi sẽ thử.\n', '\tTôi sẽ thử xem sao.\n', '\tTôi bị hói\n', '\tTôi đang bận.\n', '\tMuộn quá.\n', '\tLại đây nào.\n', '\tLại đây.\n', '\tLại đây đi.\n', '\tĐi ngủ đi.\n', '\tTôi ghét ti vi.\n', '\tTôi đã cười.\n', '\tTôi cười.\n', '\tTôi sẽ đi.\n', '\tNó có tồi không?\n', '\tLạnh.\n', '\tĐó là của chúng tôi.\n', '\tĐó là của chúng ta.\n', '\tLùi lại.\n', '\tLùi lại đi.\n', '\tLại đây.\n', '\tNó đã khóc.\n', '\tCô ấy đã khóc\n', '\tHãy ngồi ở đó.\n', '\tỞ nhà.\n', '\tGì thế?\n', '\tCó chuyện gì?\n', '\tBạn có sao không?\n', '\tHãy tìm một công việc đi.\n

In [8]:
#define the sorted list of English words
input_characters = sorted(list(input_characters))
#define the sorted list of Spanish words
target_characters = sorted(list(target_characters))
#define the number of English words
num_encoder_tokens = len(input_characters)
#define the number of Spanish words
num_decoder_tokens = len(target_characters)
#define the maximum length of the English sentences
max_encoder_seq_length = max([len(txt) for txt in input_texts])
#define the maximum length of the Spanish sentences
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs", max_decoder_seq_length)

Number of samples: 7966
Number of unique input tokens: 74
Number of unique output tokens: 163
Max sequence length for inputs: 182
Max sequence length for outputs 163


In [10]:
#index each characters in English
input_token_index = dict(
[(char, i) for i, char in enumerate(input_characters)])
#index each characters in Spanish
target_token_index = dict(
[(char, i) for i, char in enumerate(target_characters)])

In [11]:
#define the input data of the encoder as a 3-dimensional matrix populated with zeros
#the shape of the matrix is the length of the input text by the length of the max encoder by the number of encoder characters
#the np.zeros() function takes an argument of the specified dtype. Here we use float32
encoder_input_data = np.zeros(
(len(input_texts), max_encoder_seq_length, num_encoder_tokens
), dtype='float32')
#define the input data of the decoder as a 3-dimensional matrix populated with zeros
decoder_input_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens
), dtype='float32')
#define the target data of the decoder as a 3-dimensional matrix populated with zeros
decoder_target_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens
), dtype='float32')

In [12]:
#parse the input and output texts
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #define that the decoder_target_data is one time step ahead of the decoder_input_data by 
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # define the decoder_target_data to be ahead by one timestep and will not include the first character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [13]:
#define an input of the encoder with length as the number of encoder tokens
encoder_inputs = Input(shape=(None, num_encoder_tokens))
#instantiate the LSTM model
encoder = LSTM(latent_dim, return_state=True)
#define the outputs and states of the encoder
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#disregard encoder_outputs and keep only the states
encoder_states = [state_h, state_c]

In [14]:
#define an input of the encoder with length as the number of encoder tokens
decoder_inputs = Input(shape=(None, num_decoder_tokens))
#define the LSTM model for the decoder setting the return sequences and return state to True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
#define only the decoder output for the training model. The states are only needed in the inference model
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
#define the training model which requires the encoder_input_data and decoder_input_data to return the decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

Epoch 1/100
100/100 [==============================] - 13s 129ms/step - loss: 0.2519 - accuracy: 0.9272 - val_loss: 0.5433 - val_accuracy: 0.8503
Epoch 2/100
100/100 [==============================] - 13s 125ms/step - loss: 0.2507 - accuracy: 0.9274 - val_loss: 0.5457 - val_accuracy: 0.8494
Epoch 3/100
100/100 [==============================] - 13s 125ms/step - loss: 0.2492 - accuracy: 0.9279 - val_loss: 0.5390 - val_accuracy: 0.8510
Epoch 4/100
100/100 [==============================] - 13s 126ms/step - loss: 0.2478 - accuracy: 0.9282 - val_loss: 0.5420 - val_accuracy: 0.8496
Epoch 5/100
100/100 [==============================] - 13s 125ms/step - loss: 0.2469 - accuracy: 0.9284 - val_loss: 0.5410 - val_accuracy: 0.8498
Epoch 6/100
100/100 [==============================] - 13s 126ms/step - loss: 0.2455 - accuracy: 0.9289 - val_loss: 0.5384 - val_accuracy: 0.8512
Epoch 7/100
100/100 [==============================] - 13s 126ms/step - loss: 0.2441 - accuracy: 0.9293 - val_loss: 0.5365 -

In [60]:
encoder_model = Model(encoder_inputs, encoder_states)

In [61]:
# decoder_state_input_h and decoder_state_input_c are output of encoder_model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
#define the decoder input state as  a list of the hidden and cell state
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
#define the decoder output
decoder_outputs = decoder_dense(decoder_outputs)
#define the decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, 
    [decoder_outputs] + decoder_states
)

In [48]:
# Reverse-lookup token index to decode sequence back to something readable
reverse_input_char_index = dict(
(i, char) for char, i in input_token_index.items()
)
reverse_target_char_index = dict(
(i, char) for char, i in target_token_index.items()
)

In [49]:
def decode_sequence(input_seq):
    #encode the input as state vectors
    states_value = encoder_model.predict(input_seq)
    
    #generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #populate the first character of target sequence with the start character
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    #sampling loop for a batch of sequences
    #to simplify, we use batch size of 1
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value
        )
        
        #sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        #exit condition: either hit max length or find stop character
        if (len(decoded_sentence) > max_decoder_seq_length or sampled_char == '\n'):
            stop_condition = True
            
        #update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        
        #update states
        states_value = [h, c]
    
    return decoded_sentence

In [62]:
for seq_index in range(100,120):
    #take one sequence (part of the training set) for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    #call the function
    decoded_sentence = decode_sequence(input_seq)
    print()
    print(f"Input sentence: {input_texts[seq_index]}")
    print(f"Decoded sentence: {decoded_sentence}")


Input sentence: Spit it out.
Decoded sentence: Bạn đã qua cho tôi.


Input sentence: Talk slowly.
Decoded sentence: Hãy nghĩ những thứ này.


Input sentence: Talk slowly.
Decoded sentence: Hãy nghĩ những thứ này.


Input sentence: Tom saw you.
Decoded sentence: Tom đã thử lái xe đạp.


Input sentence: Tom saw you.
Decoded sentence: Tom đã thử lái xe đạp.


Input sentence: Vote for me!
Decoded sentence: Bạn đã qua cho tôi.


Input sentence: Vote for me!
Decoded sentence: Bạn đã qua cho tôi.


Input sentence: We miss Tom.
Decoded sentence: Chúng tôi đã chơi đó.


Input sentence: What's that?
Decoded sentence: Bạn đã trả lời chưa?


Input sentence: What's this?
Decoded sentence: Bạn đã trả lời chưa?


Input sentence: You're mine.
Decoded sentence: Bạn có thể làm điều đó.


Input sentence: You're mine.
Decoded sentence: Bạn có thể làm điều đó.


Input sentence: You're mine.
Decoded sentence: Bạn có thể làm điều đó.


Input sentence: Anybody home?
Decoded sentence: Bạn có thể nói cho anh ấ

In [41]:
import numpy as np

In [42]:
np.array_equal(encoder_model.get_weights()[0],model.get_weights()[0])

True

In [43]:
np.array_equal(decoder_model.get_weights()[0],model.get_weights()[3])

True

In [69]:
encoder_model.save('/content/drive/MyDrive/Colab Notebooks/encoder')
decoder_model.save('/content/drive/MyDrive/Colab Notebooks/decoder')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/encoder/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/encoder/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/decoder/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/decoder/assets
